In [2]:
import pandas as pd

In [3]:
!git clone https://github.com/stushar12/Fake-News-Detection.git

fatal: destination path 'Fake-News-Detection' already exists and is not an empty directory.


In [4]:
!unrar x "/content/Fake-News-Detection/datsaset/Fake News Detection.part1.rar"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Fake-News-Detection/datsaset/Fake News Detection.part1.rar


Would you like to replace the existing file test.csv
25144581 bytes, modified on 2019-11-29 01:52
with a new one
25144581 bytes, modified on 2019-11-29 01:52

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit 
User break

User break


In [3]:
df=pd.read_csv("/content/train.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.shape

(20800, 5)

In [7]:
###Drop Nan Values
df=df.dropna()

In [8]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df.shape

(18285, 5)

In [10]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [42]:
y=df['label']

In [43]:
y.shape

(18285,)

In [15]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
### Vocabulary size
voc_size=15000

# **Onehot Representation**

In [18]:
messages=X.copy()

In [19]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [20]:
messages.reset_index(inplace=True)

In [21]:
messages.drop(columns=["index","id","author"],axis=1,inplace=True)

In [22]:
messages

,title,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...
...,...,...
18280,Rapper T.I.: Trump a ’Poster Child For White S...,Rapper T. I. unloaded on black celebrities who...
18281,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",When the Green Bay Packers lost to the Washing...
18282,Macy’s Is Said to Receive Takeover Approach by...,The Macy’s of today grew from the union of sev...
18283,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download("all")

In [24]:
stop_words=set(stopwords.words("english"))

In [48]:
### Dataset Preprocessing

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    str1=messages['title'][i]
    str2=messages['text'][i]
    str3=str1+" "+str2
    review = re.sub('[^a-zA-Z]', ' ',str3)
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

In [51]:
corpus

18285

In [52]:
max_len=0
for i in range(0,len(corpus)):
  if(len(corpus[i].split())>max_len):
    max_len=len(corpus[i].split())

In [53]:
max_len

12065

# **One Hot Encoding**

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

# **Embedding Representation**

In [66]:
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=max_len)
print(embedded_docs)

[[    0     0     0 ...  5810 10097  4151]
 [    0     0     0 ... 11148    36  4315]
 [    0     0     0 ... 10911 13272   398]
 ...
 [    0     0     0 ...  8837  8922 13755]
 [    0     0     0 ...  4500  9329 12049]
 [    0     0     0 ...   145  2772  2156]]


In [67]:
embedded_docs[0].shape

(12065,)

In [89]:
from tensorflow.keras.layers import Dropout,BatchNormalization
## Creating model
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=max_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [90]:
len(embedded_docs),y.shape

(18285, (18285,))

In [91]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [92]:
X_final.shape,y_final.shape

((18285, 12065), (18285,))

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=0)

# **Model Training**

In [94]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=128)

Epoch 1/10
100/100 [==============================] - 94s 924ms/step - loss: 0.5139 - accuracy: 0.7665 - val_loss: 0.3332 - val_accuracy: 0.8972
Epoch 2/10
100/100 [==============================] - 93s 929ms/step - loss: 0.1751 - accuracy: 0.9357 - val_loss: 0.2100 - val_accuracy: 0.9245
Epoch 3/10
100/100 [==============================] - 93s 933ms/step - loss: 0.0889 - accuracy: 0.9693 - val_loss: 0.1919 - val_accuracy: 0.9320
Epoch 4/10
100/100 [==============================] - 93s 933ms/step - loss: 0.0426 - accuracy: 0.9875 - val_loss: 0.2080 - val_accuracy: 0.9298
Epoch 5/10
100/100 [==============================] - 93s 928ms/step - loss: 0.0229 - accuracy: 0.9936 - val_loss: 0.2345 - val_accuracy: 0.9327
Epoch 6/10
100/100 [==============================] - 93s 929ms/step - loss: 0.0124 - accuracy: 0.9968 - val_loss: 0.2700 - val_accuracy: 0.9346
Epoch 7/10
100/100 [==============================] - 93s 931ms/step - loss: 0.0152 - accuracy: 0.9951 - val_loss: 0.2694 - val_ac

# **Performance Metrics And Accuracy**

In [95]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [96]:

from sklearn.metrics import confusion_matrix

In [97]:
confusion_matrix(y_test,y_pred)

array([[2951,  133],
       [ 273, 2129]])

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9259934378417791